In [1]:
from openai import AzureOpenAI
import os
from IPython.display import  Markdown

client = AzureOpenAI(
  azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.environ.get("OPENAI_KEY"),
  api_version="2023-12-01-preview"
)

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_static_code_analysis",
            "description": "Get static code analysis of given code snippet",
            "parameters": {
                "type": "object",
                "properties": {
                    "code_block": {
                        "type": "string",
                        "description": "Code block which needs analysis"
                    },
                    "suggestions": {
                        "type": "number", 
                        "description": "limit the number of results"
                    }
                },
                "required": ["code_block"]
            }
        }
    }
]

In [4]:
def get_completion(client, messages, model="gpt-4", temperature=0, max_tokens=300, tools=None, tool_choice=None):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        tools=tools,
        tool_choice=tool_choice
    )
    return response.choices[0].message

In [11]:
messages = [
    {
        "role": "user",
        "content": """
        Code analysis for the below block of code , limit to 5 suggestions
        def sample_code():
          if true:
            print("enter")
          else:
            print("exit") 
    
        """
    }
]

In [12]:
response = get_completion(client, messages, tools=tools)
print(response)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ULK1cHzseGolEAxBAXuLLkhG', function=Function(arguments='{\n  "code_block": "def sample_code():\\n  if true:\\n    print(\\"enter\\")\\n  else:\\n    print(\\"exit\\")\\n",\n  "suggestions": 5\n}', name='get_static_code_analysis'), type='function')])


In [17]:
#sample function
def get_static_code_analysis(code_block, suggestions=10):
    """Get the static code analysis of code block"""
    result = [{
        "line": 1,
        "suggestion": "newline missing "
    },{
        "line" : 2,
        "suggestion": "Unused imports"
    }]

    return json.dumps(result)

In [13]:
get_completion(client,messages,tools=tools,tool_choice="none")

ChatCompletionMessage(content='It seems you want to analyze the provided Python code snippet and limit the suggestions to 5. I will use the static code analysis tool to perform this task. Please wait a moment while I process your request.', role='assistant', function_call=None, tool_calls=None)

In [14]:
get_completion(client,messages,tools=tools,tool_choice="auto")

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_3P9gmdH3sxSggtINmDvvKxir', function=Function(arguments='{\n  "code_block": "def sample_code():\\n  if true:\\n    print(\\"enter\\")\\n  else:\\n    print(\\"exit\\")\\n",\n  "suggestions": 5\n}', name='get_static_code_analysis'), type='function')])

In [16]:
get_completion(client,messages,tools=tools,tool_choice={"type": "function", "function": {"name": "get_static_code_analysis"}})

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jT1Fst1iyTTWOtFtHLECMDKT', function=Function(arguments='{\n  "code_block": "def sample_code():\\n  if true:\\n    print(\\"enter\\")\\n  else:\\n    print(\\"exit\\")\\n",\n  "suggestions": 5\n}', name='get_static_code_analysis'), type='function')])